In [1]:
import numpy as np 
a = [[1,2]]
a = np.append(a, [[3,4]])
a

array([1, 2, 3, 4])

In [9]:
def RichardsonExtrap(energies): 
    """
    Input: An array of energies scaled with different stretch factors--normal integer multiples only --eg 1,2,3,4..
    Output: Zero Error extrapolated answer
    """
    n = len(energies)
    
    #setting up all equations 
    equations = []
    for eq_num in range(1,n+1): 
        equation = []
        if eq_num == 1: 
            for i in range(n): 
                equation.append(1)
            equations = [equation]
        else: 
            for term_num in range(1,n+1): 
                term = term_num**(eq_num-1)
                equation.append(term)
            equations = np.append(equations, [equation], axis = 0)
    print(equations)
    
    #Now filling up equals to matrix 
    equals_to = [1]
    for i in range(1, n): 
        equals_to.append(0)
    print(equals_to)
    
    #solving the system
    coeff = np.linalg.solve(equations, equals_to)
    print(coeff)
    
    #Combine coeff with energies to get zero noise result 
    result = np.dot(coeff, energies)
    return result
            

In [10]:
a = [1,2]
print(RichardsonExtrap(a))

[[1 1]
 [1 2]]
[1, 0]
[ 2. -1.]
0.0


In [25]:
mu, sigma = 0, 0.1 # mean and standard deviation

s = np.random.normal(mu, sigma, 1000)

In [41]:
len(s)
a = []
a.append(s)
a.append(s)
a[1][0]

0.0997380711559811

In [31]:
a = [1,2]
a.append([3])
a

[1, 2, [3]]

In [30]:
s


array([ 0.09973807, -0.03498108, -0.03735706, ..., -0.00977251,
        0.03367389,  1.        ])

In [ ]:
def MonteCarloError(means, std_devs, poly_fit = False, deg = 1, sampling_size = 100): 
    """
    Input: arrays means and standard deviations of the various scalings of noise, whether to do richardson i.e. no poly fit 
    If doing polyfit, then specify degree, sampling_size is how many times to sample from the Gaussian 
    
    Assumption: Scaled by integer values ranging from 1 to X where X>1
    
    Output: Error estimate
    """
    n = len(means)
    
    #Sampling from Gaussian for each scaling 
    all_samples = []
    debug_means = []
    for i in range(n): 
        samples = np.random.normal(means[i], std_devs[i], sampling_size)
        all_samples.append(samples)
        #for debugging
        debug_means.append(means[i] - np.mean(samples))
    print(debug_means)
    
    #Doing Extrapolation for all sampling_size samples (100 if set to default)
    zero_noise_energies = []
    for j in range(sampling_size): 
        energies = []
        for i in range(n): 
            energies.append(all_samples[i][j])
            
        ##Doing either richardson or funcs 
        ###Polynomial
        if poly_fit: 
            ####need the scale values: 
            scales = []
            for s in range(1, n+1): 
                scales.append(s)
            ####making the function 
            param = np.polyfit(scales, energies, deg)
            f = np.poly1d(param)
            ####extrapolation
            result = f(0)
            zero_noise_energies.append(result)
            
        ###Richardson
        else: 
            result = RichardsonExtrap(energies)
            zero_noise_energies.append(result)
    
    #Returning error i.e. std 
    error = np.std(zero_noise_energies)
    return error
    
        
            

In [12]:
np.sqrt(10)

3.1622776601683795